## Introduction
### This notebook implements data preprocessing for the text data. In the following cell, we import required packages.

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd # for reading and manipulating the dataframes. 
from bs4 import BeautifulSoup # This is a packages parsing html tags 
import re # A powerful tool for matching patterns in strings The re module allows to perform a variety of operations,
#such as searching, splitting, and substituting parts of strings based on patterns.
from sklearn.preprocessing import LabelEncoder # for encoding text to uinuqe values
from sklearn.feature_selection import mutual_info_classif # for measuring the mutual information between two components
from sklearn.feature_extraction.text import CountVectorizer # to covert text to embeding
import numpy as np
import matplotlib.pyplot as plt

# Reading train and validation sets
df_list =[ pd.read_csv("train.csv") ,pd.read_csv("test.csv")]
df_list[0].columns = ['label', 'q1', 'q2', 'Ans']
df_list[1].columns = ['label', 'q1', 'q2', 'Ans']
print(df_list[0].keys())

for i in range(len(df_list)):
    df_list[i] = df_list[i].dropna()

Index(['label', 'q1', 'q2', 'Ans'], dtype='object')


### Data Preprocessing

#### Following imported the packages we apply a set of preprocessing operation as follows.

 1- Removing html tags, \t and \n and double spaces. As can be seen in the dataset, unrequired characters are htmls tags, tabs, new lines, and double spaces. 

 2- Merging q1,q2, and Ans coulmns into new column named Text and remove the previous one columns. 


In [2]:
# This functions cleans the text by removing HTML tags, tabs, new lines, and double spaces.
def text_cleaning(text):
    if pd.isna(text):
        return ""
    try:
        # Remove HTML tags
        soup = BeautifulSoup(text, "lxml")
        text = soup.get_text()  # returns text without the HTML tags.
    except Exception as e:
        print(f"Error parsing text: {e}")
        return ""
    # Removing tab, newline, and double spaces
    text = re.sub(r'[\t\n\s]+', ' ', text)
    # Stripping leading and trailing spaces
    text = text.strip()
    return text

# Over two dataframes, i.e., train and validation datasets, we apply function text_cleaning on each of the columns Title and Body
columns = ['q1','q2','Ans']
for df in df_list:
    for column in columns:
        df[column] = df[column].apply(text_cleaning)
        
# List of columns should be drop
columns_to_drop = ['q1','q2','Ans']
# when adding new columns as result of merging 'Title' and 'Body', the new columns is added in the lasy layer. We would like to move
#label column named as 'Y' to the last column
column_to_move = 'label'
# we create an object from the following class to apply encoding.
for index,df in enumerate(df_list): #This loop goes over two dataframes
    df['Text'] = df['q1'] + " " + df['q2']+ " " + df['Ans'] # Merging two columns
    df = df[[col for col in df.columns if col != column_to_move] + [column_to_move]] # Reordering the columns
    df.drop(columns=columns_to_drop, axis=1, inplace=True) # Dropping the columns 'q1','q2','Ans'
    df['label'] = df['label'] - 1
    print(df['label'].max(),df['label'].min())
    df_list[index] = df

# prininting the columns name of each dataframe to check the correctness of the previous operations
for df in df_list:
    print(df.keys())

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



Error parsing text: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?
Error parsing text: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?
Error parsing text: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?
Error parsing text: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?
Error parsing text: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?
Error parsing text: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?
Error parsing text: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?
Error parsing text: Couldn't find a tree builder with the features you requested: lxml. Do you ne

### Check for data imblacne issue

As the last preprocessing step, we count number of samples per class, to see if there is any imbalanc issue in the data.

In [ ]:
class_counts = df_list[0]['label'].value_counts()
x = np.arange(len(class_counts))
plt.bar(x,class_counts)
plt.xlabel('Classes')
plt.ylabel('Number of Samples')
print(class_counts)

### Saving the resulted Dataframes

In [ ]:
df_list[0].to_csv('preprocessed_train.csv', index=False)
df_list[1].to_csv('preprocessed_test.csv', index=False)